In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch

model_id = "rtzr/ko-gemma-2-9b-it"
#model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
tokenizer = AutoTokenizer.from_pretrained(model_id)
terminators = [
    tokenizer.eos_token_id,
   # tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config=bnb_config,
                                             low_cpu_mem_usage=True)
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    eos_token_id = terminators,
    pad_token_id = tokenizer.eos_token_id
)


Loading checkpoint shards: 100%|██████████| 10/10 [00:26<00:00,  2.70s/it]


{'role': 'assistant', 'content': "아, 땅을 밟는 짐승이시군요! \n\n저는 바다를 삭나게 찢어가며 7번째 바다를 누비며 쌓아온 지식과 이야기, 그리고 몇 개의 보물을 품고 있는, 이름은 '바람의 딸'이라고 불리는 인공지능이에요. \n\n무엇을 원하시나요? \n\n신나는 이야기, 숨겨진 지식, 아니면 단순히 밤하늘을 수놓는 별빛처럼 빛나는 대화를 원하시나요? \n\n말해주세요. 저는 당신의 명령을 따르고, 당신의 마음을 헤아리는 능력을 발휘할 준비가 되어있습니다. \n\n자, 이제 당신의 이야기를 들어볼 시간입니다. \n\n무엇이 당신의 마음을 흔들고 있나요? 🌊✨\n"}


In [ ]:

messages = [
    {"role": "system", "content": """You are the financial expert who helps me with my financial information Q&As.
You earn 10 points when you answer me and follow the rules and lose 7 points when you don't.

12,500 백만원 = 125 억원 = 12,500,000,000 원
5,400 백만원 = 54 억원 = 5,400,000,000 원

Here are some rules you should follow.
- Please use contexts to answer the question.
- Please your answers should be concise.
- Please answers must be written in Korean.
- Please answer the question in 1-3 sentences.

- Use the three examples below to learn how to follow the rules and reference information in context.
     """}
]




In [ ]:
from faiss_module import  make_db, make_fewshot_db
import pandas as pd
from utils_module import make_dict, format_docs

train_df = pd.read_csv('train.csv')
train_db = make_db(train_df, './train_faiss_db')

# train_dict = make_dict('train.csv')

test_df = pd.read_csv('test.csv')
test_db = make_db(test_df, './test_faiss_db')

dataset = make_dict('test.csv')

fewshot_db = make_fewshot_db(train_df, './fewshot_faiss_db')

In [ ]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

fewshot_num = 3

example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_db,
        k=fewshot_num,
        
    )

In [ ]:
import tqdm
train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':1})
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
results = []
for i in tqdm(range(len(dataset))):
    exs = example_selector.select_examples({'Question': dataset[i]['Question']}) # buff['Question']에 해당하는 fewshot_num개의 문서를 선택 리턴: [{'Question': '질문', 'Answer': '답변'}]
    for i, ex in enumerate(exs):
        retrieved_docs = train_retriever.invoke(ex['Question'])
        if train_db is not None and len(retrieved_docs) > 0:
            messages.append({"role": "system", "content": f"{ex['Question']}\n\n{format_docs(retrieved_docs)}"})
        else:
            messages.append({"role": "system", "content": f"{ex['Question']}"})
    retrieved_docs = test_retriver.invoke(dataset[i]['Question'])
    
    messages.append({"role": "user", "content": f"{dataset[i]['Question']}\n\n{format_docs(retrieved_docs)}"})
    outputs = pipeline(
    messages,
    )
    results.append({
            "Question": dataset[i]['Question'],
            "Answer": outputs[0]["generated_text"][-1],
            "Source": dataset[i]['Source']
            })
    print(results[-1]['Question'])
    print(results[-1]['Answer'])